In [ ]:
from TauLidarCommon.frame import FrameType, Frame
from TauLidarCamera.camera import Camera
from TauLidarCamera.constants import VALUE_20MHZ
from TauLidarCommon.color import ColorMode

In [ ]:
camera = Camera.open()

In [ ]:
cameraInfo = camera.info()

print("\nToF camera opened successfully:")
print("    model:      %s" % cameraInfo.model)
print("    firmware:   %s" % cameraInfo.firmware)
print("    uid:        %s" % cameraInfo.uid)
print("    resolution: %s" % cameraInfo.resolution)
print("    port:       %s" % cameraInfo.port)

In [ ]:
camera.setModulationChannel(0) ## autoChannelEnabled: 0, channel: 0

In [ ]:
camera.setIntegrationTime3d(0, 800)  ## set integration time 0: 1000
camera.setMinimalAmplitude(0, 60)  ## set minimal amplitude 0: 80
Camera.setRange(0, 4500) ## points in the distance range to be colored
camera.setIntegrationTimeGrayscale(15000)  ## set integration time grayscale: 8000, needed when requiring FrameType.DISTANCE_GRAYSCALE

In [ ]:
frame = camera.readFrame(FrameType.DISTANCE)
frame = camera.readFrame(FrameType.DISTANCE_GRAYSCALE)
frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)

In [ ]:
frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)

In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

TESTING WITH YOLOV5s

In [ ]:
# Using the YOLOv8s model
model = YOLO("yolov8s.pt")

# If you have the custom model. Provide the absolute path to run the custom model
# path = "C://Users//jaura//Downloads//runs//detect//yolov8s_custom11//weights//best.pt"
# model = YOLO(path)

In [ ]:
import random
# Start capturing video
try:
    while True:
        # Read a frame from the camera
        frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)
        mat_amplitude = np.frombuffer(frame.data_amplitude, dtype=np.float32, count=-1, offset=0).reshape(frame.height, frame.width)

        # Convert to 8-bit for display
        # Normalize the image for better visualization
        cv2.normalize(mat_amplitude, mat_amplitude, 0, 255, cv2.NORM_MINMAX)
        mat_amplitude = mat_amplitude.astype(np.uint8)

        # Apply a colormap to the image
        colored_image = cv2.applyColorMap(mat_amplitude, cv2.COLORMAP_JET)

        # Perform object detection
        results = model.predict(source=colored_image, conf=0.5)  # Confidence threshold

        # Define a list of colors for different classes
        colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(model.names))]

        for result in results:
            boxes = result.boxes  # Bounding box predictions
            for box in boxes:
                # Convert tensor to a NumPy array and extract coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

                # Extract confidence and class ID
                conf = box.conf[0].item()  # Confidence score
                cls = int(box.cls[0].item())  # Class ID

                # Generate the label
                label = f"{model.names[cls]}: {conf:.2f}"

                # Get a unique color for the class
                box_color = colors[cls]
                text_color = (255 - box_color[0], 255 - box_color[1], 255 - box_color[2])  # Invert box color for text

                # Draw the box and label on the image
                cv2.rectangle(colored_image, (x1, y1), (x2, y2), box_color, 2)  # Box with unique color
                cv2.putText(colored_image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1)  # Smaller font size


        # Enlarge the image while preserving aspect ratio
        target_width = 1200  # Desired width
        target_height = 1000  # Desired height
        resized_image = cv2.resize(colored_image, (target_width, target_height), interpolation=cv2.INTER_LINEAR)

        # Display the colored image with bounding boxes
        cv2.imshow('Object Detection on Depth Map', resized_image)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Release resources and close the window
    cv2.destroyAllWindows()